In [1]:
"""def generate_synthetic_samples_imbalanced(X, y, n_samples, threshold=0.2, imbalance_factor=2, random_state=42):
    np.random.seed(random_state)
    nn = NearestNeighbors(n_neighbors=5)
    nn.fit(X)

    synthetic_X = []
    synthetic_y = []

    # Calculate the distance to the nearest neighbors
    distances, neighbors = nn.kneighbors(X)

    # Identify samples with fewer neighbors within the threshold distance
    low_density_indices = [i for i, dist in enumerate(distances[:, 1:]) if np.sum(dist < threshold) < 3]

    # Increase the duplication factor for imbalanced samples
    for _ in range(n_samples):
        # Select a sample, with preference to low-density samples
        if low_density_indices and np.random.rand() < 0.5:
            idx = np.random.choice(low_density_indices)
        else:
            idx = np.random.randint(0, len(X))

        neighbor_idxs = neighbors[idx]
        neighbor_idx = np.random.choice(neighbor_idxs)

        # Linear interpolation for synthetic sample generation
        lam = np.random.uniform(0, 1)
        new_sample_X = X[idx] + lam * (X[neighbor_idx] - X[idx])
        new_sample_y = y[idx] + lam * (y[neighbor_idx] - y[idx])

        synthetic_X.append(new_sample_X)
        synthetic_y.append(new_sample_y)

        # Additional synthetic samples for imbalanced samples
        if idx in low_density_indices:
            for _ in range(imbalance_factor - 1):
                neighbor_idx = np.random.choice(neighbor_idxs)
                lam = np.random.uniform(0, 1)
                new_sample_X = X[idx] + lam * (X[neighbor_idx] - X[idx])
                new_sample_y = y[idx] + lam * (y[neighbor_idx] - y[idx])

                synthetic_X.append(new_sample_X)
                synthetic_y.append(new_sample_y)

    return np.array(synthetic_X), np.array(synthetic_y)"""

'def generate_synthetic_samples_imbalanced(X, y, n_samples, threshold=0.2, imbalance_factor=2, random_state=42):\n    np.random.seed(random_state)\n    nn = NearestNeighbors(n_neighbors=5)\n    nn.fit(X)\n\n    synthetic_X = []\n    synthetic_y = []\n\n    # Calculate the distance to the nearest neighbors\n    distances, neighbors = nn.kneighbors(X)\n\n    # Identify samples with fewer neighbors within the threshold distance\n    low_density_indices = [i for i, dist in enumerate(distances[:, 1:]) if np.sum(dist < threshold) < 3]\n\n    # Increase the duplication factor for imbalanced samples\n    for _ in range(n_samples):\n        # Select a sample, with preference to low-density samples\n        if low_density_indices and np.random.rand() < 0.5:\n            idx = np.random.choice(low_density_indices)\n        else:\n            idx = np.random.randint(0, len(X))\n\n        neighbor_idxs = neighbors[idx]\n        neighbor_idx = np.random.choice(neighbor_idxs)\n\n        # Linear inte

In [2]:
"""# Optimise test size and number of added samples
X = df_vape[['tax_increase', 'outlet_reduction', 'dec_smoking_prevalence', 
              'dec_tobacco_supply', 'dec_smoking_uptake', 'average_age', 
              'gender_idx', 'ethnicity_idx']]
y = df_vape[['qalys_pc']]

# Ensure that y is a 1D array for compatibility
y_flat = y.values.flatten()

# Hyperparameters to test
test_sizes = [0.2, 0.25, 0.3, 0.35, 0.4, 0.45]
added_samples = [100, 150, 200, 250, 300]

# Best MAPE and hyperparameter values to use
rfnb_best_test_size = 0
rfnb_best_sam = 0
rfnb_best_mape = 1e6

rfb_best_test_size = 0
rfb_best_sam = 0
rfb_best_mape = 1e6

xgb_best_test_size = 0
xgb_best_sam = 0
xgb_best_mape = 1e6

for test_size in test_sizes:
    for n_samples in added_samples:
        # Generate synthetic samples with fixed random state
        X_res, y_res = generate_synthetic_samples(X.values, y_flat, n_samples=n_samples, random_state=42)
        
        # Stack the original and synthetic data
        X_full = np.vstack([X.values, X_res])
        y_full = np.hstack([y_flat, y_res])
        
        # Convert to DataFrame for easier handling
        df_resampled = pd.DataFrame(X_full, columns=X.columns)
        df_resampled['qalys_pc'] = y_full
        
        X_resampled = df_resampled[['tax_increase', 'outlet_reduction', 'dec_smoking_prevalence', 
              'dec_tobacco_supply', 'dec_smoking_uptake', 'average_age', 
              'gender_idx', 'ethnicity_idx']]
        y_resampled = df_resampled[['qalys_pc']]

        print(f'Test Size: {test_size}, Samples added: {n_samples}')
        _, rfnb_mape = rf_no_bootstrap(X_resampled, y_resampled, test_size)
        _, rfb_mape = rf_bootstrap(X_resampled, y_resampled, test_size)
        _, xgb_mape = xgboost(X_resampled, y_resampled, test_size)
        print("-----------------------------")

        if rfnb_mape < rfnb_best_mape:
            rfnb_best_mape = rfnb_mape
            rfnb_best_sam = n_samples
            rfnb_best_test_size = test_size

        if rfb_mape < rfb_best_mape:
            rfb_best_mape = rfb_mape
            rfb_best_sam = n_samples
            rfb_best_test_size = test_size

        if xgb_mape < xgb_best_mape:
            xgb_best_mape = xgb_mape
            xgb_best_sam = n_samples
            xgb_best_test_size = test_size

print("-----------------------------")
print("")
print(f'Random Forest without Bootstrapping - Best MAPE: {rfnb_best_mape}, Best number of samples to add: {rfnb_best_sam}, Best Test Size: {rfnb_best_test_size}')
print(f'Random Forest with Bootstrapping - Best MAPE: {rfb_best_mape}, Best number of samples to add: {rfb_best_sam}, Best Test Size: {rfb_best_test_size}')
print(f'XGBoost - Best MAPE: {xgb_best_mape}, Best number of samples to add: {xgb_best_sam}, Best Test Size: {xgb_best_test_size}')"""

'# Optimise test size and number of added samples\nX = df_vape[[\'tax_increase\', \'outlet_reduction\', \'dec_smoking_prevalence\', \n              \'dec_tobacco_supply\', \'dec_smoking_uptake\', \'average_age\', \n              \'gender_idx\', \'ethnicity_idx\']]\ny = df_vape[[\'qalys_pc\']]\n\n# Ensure that y is a 1D array for compatibility\ny_flat = y.values.flatten()\n\n# Hyperparameters to test\ntest_sizes = [0.2, 0.25, 0.3, 0.35, 0.4, 0.45]\nadded_samples = [100, 150, 200, 250, 300]\n\n# Best MAPE and hyperparameter values to use\nrfnb_best_test_size = 0\nrfnb_best_sam = 0\nrfnb_best_mape = 1e6\n\nrfb_best_test_size = 0\nrfb_best_sam = 0\nrfb_best_mape = 1e6\n\nxgb_best_test_size = 0\nxgb_best_sam = 0\nxgb_best_mape = 1e6\n\nfor test_size in test_sizes:\n    for n_samples in added_samples:\n        # Generate synthetic samples with fixed random state\n        X_res, y_res = generate_synthetic_samples(X.values, y_flat, n_samples=n_samples, random_state=42)\n        \n        # Sta